In [29]:
import pandas as pd
import pickle
import numpy as np
import networkx as nx
import re
import time

In [84]:
# df_ivs = pickle.load(open('data/ivs_exploded_100.p', 'rb'))
# df_ivs_old = pickle.load(open('data/ivs_exploded_100_old.p', 'rb'))
# routes_dests = pickle.load(open('data/users_ship_specific_routes.p', 'rb'))
# df_ships = pd.read_excel('data/ship_types.xlsx')

# G = pickle.load(open("data/network.p", "rb"))
# paths = pickle.load(open("data/paths.p", "rb"))
# df_random = pickle.load(open("data/df_random.p", "rb"))
# df_abm = pickle.load(open("data/df_abm.p", "rb"))
# non_zero_flows = pickle.load(open("data/non_zero_flows.p", "rb"))
# feasible_combinations = pickle.load(open('data/feasible_comb.p', 'rb'))
# optimal_flows = pickle.load(open('data/optimal_flows.p', 'rb'))
# path_lengths = pickle.load(open('data/path_lengths_ship_specific_routes.p', 'rb'))
agent_data = pickle.load(open('data/agent_data.p', 'rb'))
model_data = pickle.load(open('data/model_data.p', 'rb'))

In [31]:
a = pd.DataFrame.from_dict(model_data.loc[1440].item()).sort_values(by='id')
# b = a.loc[a.time_departed>(24*60)]
a['pure_travel_time'] = a['travel_time'] - a['time_in_line'] - a['time_charging']
b = a.loc[a.route==('NLGOR', 'NLRTM', 0)]

In [32]:
import pandas as pd; 
pd.set_option('max_colwidth', 800)

In [33]:
a= a.sort_values('route')
median_dict = a.groupby('route').median().loc[:,['pure_travel_time']].to_dict()['pure_travel_time']

In [34]:
a['median_route'] = a.route.apply(lambda x: median_dict[x])

In [35]:
a.loc[a.median_route != a.pure_travel_time]

,id,route,time_departed,travel_time,time_in_line,time_charging,time_charging_dest,full_charging_info,distance_travelled,battery_size,pure_travel_time,median_route
76,74,"(NLVEN, NLBON, 0)",1044,335,0,93,0,{'134': 93},60694.927509,4750,242,242.5
55,51,"(NLVEN, NLBON, 0)",672,346,0,103,0,{'134': 103},60694.927509,4750,243,242.5
19,7,"(NLVEN, NLBON, 0)",61,335,0,93,0,{'134': 93},60694.927509,4750,242,242.5
40,39,"(NLVEN, NLBON, 0)",496,346,0,103,0,{'134': 103},60694.927509,4750,243,242.5


In [36]:
a.loc[a.route == ('NLFAR', 'NLEEM', 0)]

,id,route,time_departed,travel_time,time_in_line,time_charging,time_charging_dest,full_charging_info,distance_travelled,battery_size,pure_travel_time,median_route


## Process agent data

### Vessel data

In [85]:
agent_data = agent_data.reset_index()
vessel_data = agent_data.loc[agent_data.vessel_status.notnull()]
vessel_data = vessel_data.drop(columns=['removed_at'])
vessel_data['status'] = 0
vessel_data['at_station'] = 0
vessel_data.status = vessel_data.vessel_status.apply(lambda x: x[0] if type(x) == list else x)
vessel_data.at_station = vessel_data.vessel_status.apply(lambda x: x[1] if type(x) == list else x)

In [86]:
vessel_data_dict = {}
def process_vessel_data(x):
    if not x.AgentID in vessel_data_dict.keys():
        vessel_data_dict[x.AgentID] = {"charging":{}, "inline":{}, "driving":0}
    if x.status == 'driving':
        vessel_data_dict[x.AgentID]['driving'] += 1
    else:
        if not x.at_station in vessel_data_dict[x.AgentID][x.status].keys():
            vessel_data_dict[x.AgentID][x.status][x.at_station] = 1
        else:
            vessel_data_dict[x.AgentID][x.status][x.at_station] += 1

In [87]:
vessel_data.apply(lambda x: process_vessel_data(x), axis=1)

3402      None
3646      None
3890      None
4134      None
4378      None
          ... 
371840    None
371841    None
371842    None
371843    None
371844    None
Length: 21682, dtype: object

In [88]:
df = pd.DataFrame(vessel_data_dict)
df = df.transpose()

In [89]:
df['total_charging'] = 0
df.total_charging = df.charging.apply(lambda x: sum(x.values()))
df['total_inline'] = 0
df.total_inline = df.inline.apply(lambda x: sum(x.values()))

In [91]:
df1 = vessel_data.loc[vessel_data.status != 0].loc[:,['AgentID','Step','vessel_route','battery_size']].groupby('AgentID').first()
vessel_data = pd.merge(df1, df, left_index=True, right_index=True)

In [92]:
vessel_data
# looks okay! Final check with pycharm later

,Step,vessel_route,battery_size,charging,inline,driving,total_charging,total_inline
AgentID,,,,,,,,
0,13,"(NLGOR, NLRTM, 0)",2300.0,{'160': 41},{},237,41,0
1,33,"(NLAMS, NLZAA, 0)",1450.0,{},{},30,0,0
2,33,"(NLMOE, NLAPN, 1)",4750.0,"{'159': 85, '8865003': 114}",{},315,199,0
3,35,"(NLSWI, NLLEY, 0)",1450.0,{},{},99,0,0
4,37,"(NLKGZ, NLZAA, 0)",4750.0,{},{},22,0,0
...,...,...,...,...,...,...,...,...
93,1359,"(NLNIJ, NLABL, 0)",4750.0,{},{},82,0,0
94,1402,"(NLAMS, NLZAA, 0)",2300.0,{},{},30,0,0
95,1422,"(NLNIJ, NLWAS, 0)",4750.0,{},{},19,0,0


### Charging station data

In [61]:
df1 = pd.DataFrame(agent_data.loc[(agent_data.charging_stations>0)].groupby('AgentID').first().loc[:,'charging_stations'])

In [62]:
df2 = pd.DataFrame(agent_data.loc[(agent_data.charging_stations>0)].groupby('AgentID').station_status.mean())

In [81]:
cs_data = pd.merge(df1, df2, left_index=True, right_index=True)

In [83]:
cs_data

,charging_stations,station_status
AgentID,,
112,1.0,0.203331
113,1.0,0.172103
116,1.0,0.208883
122,1.0,0.020125
124,1.0,0.000000
125,1.0,0.018737
132,1.0,0.120749
134,1.0,0.272033
137,1.0,0.076336


## Old from here

In [4]:
agent_data = agent_data_original.reset_index()
agent_data = agent_data.drop(columns=['removed_at'])
agent_data['status'] = 0
agent_data['at_station'] = 0
agent_data.status = agent_data.vessel_status.apply(lambda x: x[0] if type(x) == list else x)
agent_data.at_station = agent_data.vessel_status.apply(lambda x: x[1] if type(x) == list else x)

# make new df with vessels only and prepare to store this data
vessel_df = agent_data.loc[agent_data.status!=0]
vessel_df1 = vessel_df.groupby('AgentID').first().drop(columns=['vessel_status', 'battery_level', 'station_status']).reset_index()
vessel_df1['charging'] = 0
vessel_df1['inline'] = 0
vessel_df1['driving'] = 0
vessel_df1['detailed_charging'] = 0
vessel_df1['detailed_inline'] = 0

# only keep vessels
vessel_df2 = vessel_df.groupby(['AgentID', 'status', 'at_station']).count()
vessel_df2 = vessel_df2.reset_index()

# also extract non_detailed charging and waiting data
d = vessel_df2.copy()
charging_times = d.loc[d.status=='charging'].loc[:,['Step']].to_dict()['Step']
driving_times = d.loc[d.status=='driving'].loc[:,['Step']].to_dict()['Step']
inline_times = d.loc[d.status=='inline'].loc[:,['Step']].to_dict()['Step']

vessel_df1['charging'] = vessel_df1.AgentID.apply(lambda x: charging_times[x] if x in charging_times.keys() else 0)
vessel_df1['driving'] = vessel_df1.AgentID.apply(lambda x: driving_times[x] if x in driving_times.keys() else 0)
vessel_df1['inline'] = vessel_df1.AgentID.apply(lambda x: inline_times[x] if x in inline_times.keys() else 0)

# also extract full charging and waiting data
vessel_df2.apply(lambda x: process_agent_data(x), axis=1)

#add this data to the df too
vessel_df1['detailed_charging'] = vessel_df1.apply(lambda x: agent_data_dict[x.AgentID]['charging'], axis=1)
vessel_df1['detailed_inline'] = vessel_df1.apply(lambda x: agent_data_dict[x.AgentID]['inline'], axis=1)


In [7]:
driving_times

{0: 121,
 2: 228,
 3: 130,
 6: 278,
 10: 579,
 12: 184,
 13: 127,
 15: 228,
 17: 188,
 19: 184,
 22: 206,
 23: 30,
 24: 148,
 25: 127,
 27: 228,
 29: 87,
 34: 663,
 35: 143,
 37: 228,
 42: 567,
 43: 99,
 44: 148,
 46: 195,
 48: 228,
 50: 187,
 52: 228,
 55: 155,
 56: 158,
 57: 183,
 58: 121,
 60: 206,
 63: 275,
 65: 155,
 66: 183,
 67: 22,
 68: 110,
 70: 228,
 72: 110,
 75: 315,
 78: 255,
 79: 148,
 80: 65,
 81: 188,
 82: 188,
 84: 228,
 86: 183,
 87: 22,
 88: 110,
 89: 30,
 91: 155,
 94: 315,
 97: 184,
 98: 13,
 100: 206,
 102: 275,
 103: 110,
 105: 275,
 107: 148,
 108: 65,
 111: 315,
 112: 148,
 113: 30,
 114: 130,
 116: 228,
 119: 155,
 123: 453,
 125: 237,
 126: 188,
 128: 184,
 130: 184,
 134: 315,
 136: 13,
 137: 121,
 140: 278,
 141: 158,
 144: 348,
 149: 379,
 150: 121,
 151: 127,
 154: 407,
 156: 158,
 157: 87,
 158: 188,
 160: 267,
 163: 187,
 164: 110,
 165: 130,
 166: 30,
 168: 184,
 169: 65,
 170: 183,
 171: 30,
 173: 255,
 175: 231,
 178: 270,
 181: 246,
 183: 206,
 186:

In [5]:
vessel_df1

,AgentID,Step,vessel_route,battery_size,generated_at,status,at_station,charging,inline,driving,detailed_charging,detailed_inline
0,0,8,"(NLTNZ, NLBRS, 0)",2300,7,driving,driving,0,0,121,{},{}
1,1,21,"(NLNIJ, NLWAS, 0)",4750,20,driving,driving,88,0,0,{'8865600': 88},{}
2,2,32,"(NLDOR, NLRTM, 0)",4750,31,driving,driving,0,0,228,{},{}
3,3,84,"(NLAMS, NLUTC, 1)",4750,83,driving,driving,0,0,130,"{'132': 100, '8864837': 37}",{}
4,4,84,"(NLNIJ, NLABL, 0)",6717,83,driving,driving,100,0,0,"{'116': 143, '160': 148, '8865412': 145}",{}
...,...,...,...,...,...,...,...,...,...,...,...,...
114,114,1351,"(NLKGZ, NLAMS, 0)",1000,1350,driving,driving,0,0,130,{},{}
115,115,1398,"(NLGOR, NLRTM, 0)",2300,1397,driving,driving,99,0,0,{},{}
116,116,1400,"(NLKGZ, NLAMS, 0)",2300,1399,driving,driving,0,0,228,{},{}
117,117,1401,"(NLNIJ, NLABL, 0)",4750,1400,driving,driving,0,132,0,{},{}


In [6]:
vessel_df2

,AgentID,status,at_station,Step,vessel_status,vessel_route,battery_size,battery_level,generated_at,station_status
0,0,driving,driving,121,121,121,121,121,121,0
1,1,charging,8865600,88,88,88,88,88,88,0
2,1,driving,driving,228,228,228,228,228,228,0
3,2,driving,driving,130,130,130,130,130,130,0
4,3,charging,132,100,100,100,100,100,100,0
...,...,...,...,...,...,...,...,...,...,...
217,114,driving,driving,13,13,13,13,13,13,0
218,115,driving,driving,43,43,43,43,43,43,0
219,116,driving,driving,13,13,13,13,13,13,0
220,117,driving,driving,40,40,40,40,40,40,0


In [11]:
agent_data_dict

{0: {'charging': {}, 'inline': {}, 'driving': 121},
 1: {'charging': {'8865600': 88}, 'inline': {}, 'driving': 228},
 2: {'charging': {}, 'inline': {}, 'driving': 130},
 3: {'charging': {'132': 100, '8864837': 37}, 'inline': {}, 'driving': 278},
 4: {'charging': {'116': 143, '160': 148, '8865412': 145},
  'inline': {},
  'driving': 579},
 5: {'charging': {'8867538': 55}, 'inline': {}, 'driving': 184},
 6: {'charging': {}, 'inline': {}, 'driving': 127},
 7: {'charging': {'8865600': 99}, 'inline': {'8865600': 60}, 'driving': 228},
 8: {'charging': {}, 'inline': {}, 'driving': 188},
 9: {'charging': {'8867538': 107}, 'inline': {'8867538': 44}, 'driving': 184},
 10: {'charging': {'113': 167}, 'inline': {}, 'driving': 206},
 11: {'charging': {}, 'inline': {}, 'driving': 30},
 12: {'charging': {}, 'inline': {}, 'driving': 148},
 13: {'charging': {}, 'inline': {}, 'driving': 127},
 14: {'charging': {'8865600': 87}, 'inline': {'8865600': 128}, 'driving': 228},
 15: {'charging': {}, 'inline': {

In [51]:
df_temp = agent_data.loc[agent_data.status!=0]
df_temp = df_temp.groupby(['AgentID', 'status', 'at_station']).count()

In [47]:
df_temp

Step  vessel_status  vessel_route  battery_size  \
AgentID status   at_station                                                    
0       driving  driving      121            121           121           121   
1       charging 8865600       88             88            88            88   
        driving  driving      228            228           228           228   
2       driving  driving      130            130           130           130   
3       charging 132          100            100           100           100   
...                           ...            ...           ...           ...   
114     driving  driving       13             13            13            13   
115     driving  driving       43             43            43            43   
116     driving  driving       13             13            13            13   
117     driving  driving       40             40            40            40   
118     driving  driving       39             39            39            39   

                             battery_level  generated_at  removed_at  \
AgentID status   at_station                                            
0       driving  driving               121           121           0   
1       charging 8865600                88            88           0   
        driving  driving               228           228           0   
2       driving  driving               130           130           0   
3       charging 132                   100           100           0   
...                                    ...           ...         ...   
114     driving  driving                13            13           0   
115     driving  driving                43            43           0   
116     driving  driving                13            13           0   
117     driving  driving                40            40           0   
118     driving  driving                39            39           0   

                             station_status  
AgentID status   at_station                  
0       driving  driving                  0  
1       charging 8865600                  0  
        driving  driving                  0  
2       driving  driving                  0  
3       charging 132                      0  
...                                     ...  
114     driving  driving                  0  
115     driving  driving                  0  
116     driving  driving                  0  
117     driving  driving                  0  
118     driving  driving                  0  

[222 rows x 8 columns]

In [12]:
e

,AgentID,status,at_station,Step,vessel_status,vessel_route,battery_size,battery_level,generated_at,removed_at,station_status
0,0,driving,driving,121,121,121,121,121,121,0,0
1,1,charging,8865600,88,88,88,88,88,88,0,0
2,1,driving,driving,228,228,228,228,228,228,0,0
3,2,driving,driving,130,130,130,130,130,130,0,0
4,3,charging,132,100,100,100,100,100,100,0,0
...,...,...,...,...,...,...,...,...,...,...,...
460,8868191,0,0,1440,1440,1440,1440,1440,1440,1440,1440
461,8868208,0,0,1440,1440,1440,1440,1440,1440,1440,1440
462,8868222,0,0,1440,1440,1440,1440,1440,1440,1440,0
463,8868266,0,0,1440,1440,1440,1440,1440,1440,1440,0


In [13]:
agent_data_dict = {}
e = d.reset_index()
e = e.loc[e.status!=0]
e.apply(lambda x: process_agent_data(x), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
217    None
218    None
219    None
220    None
221    None
Length: 222, dtype: object

In [21]:
df_test = pd.DataFrame(agent_data_dict)
df_test.transpose()

,charging,inline,driving
0,{},{},121
1,{'8865600': 88},{},228
2,{},{},130
3,"{'132': 100, '8864837': 37}",{},278
4,"{'116': 143, '160': 148, '8865412': 145}",{},578
...,...,...,...
114,{},{},13
115,{},{},43
116,{},{},13
117,{},{},40


In [48]:
d = d.reset_index()

In [49]:
d = d.set_index('AgentID')

In [50]:
d.loc[d.status=='charging']

,status,at_station,Step,vessel_status,vessel_route,battery_size,battery_level,generated_at,removed_at,station_status
AgentID,,,,,,,,,,
1,charging,8865600,88,88,88,88,88,88,0,0
3,charging,132,100,100,100,100,100,100,0,0
3,charging,8864837,37,37,37,37,37,37,0,0
4,charging,116,143,143,143,143,143,143,0,0
4,charging,160,148,148,148,148,148,148,0,0
...,...,...,...,...,...,...,...,...,...,...
106,charging,144,73,73,73,73,73,73,0,0
107,charging,134,6,6,6,6,6,6,0,0
109,charging,8866686,95,95,95,95,95,95,0,0


In [50]:
vessel_df1

,AgentID,Step,vessel_route,battery_size,generated_at,status,at_station,charging,inline,driving
0,0,8,"(NLTNZ, NLBRS, 0)",2300,7,driving,driving,0,0,0
1,1,21,"(NLNIJ, NLWAS, 0)",4750,20,driving,driving,0,0,0
2,2,32,"(NLDOR, NLRTM, 0)",4750,31,driving,driving,0,0,0
3,3,84,"(NLAMS, NLUTC, 1)",4750,83,driving,driving,0,0,0
4,4,84,"(NLNIJ, NLABL, 0)",6717,83,driving,driving,0,0,0
...,...,...,...,...,...,...,...,...,...,...
114,114,1351,"(NLKGZ, NLAMS, 0)",1000,1350,driving,driving,0,0,0
115,115,1398,"(NLGOR, NLRTM, 0)",2300,1397,driving,driving,0,0,0
116,116,1400,"(NLKGZ, NLAMS, 0)",2300,1399,driving,driving,0,0,0
117,117,1401,"(NLNIJ, NLABL, 0)",4750,1400,driving,driving,0,0,0


In [51]:
charging_times = d.loc[d.status=='charging'].loc[:,['Step']].to_dict()['Step']
driving_times = d.loc[d.status=='driving'].loc[:,['Step']].to_dict()['Step']
inline_times = d.loc[d.status=='inline'].loc[:,['Step']].to_dict()['Step']

main_agent_df['charging'] = main_agent_df.AgentID.apply(lambda x: charging_times[x] if x in charging_times.keys() else 0)
main_agent_df['driving'] = main_agent_df.AgentID.apply(lambda x: driving_times[x] if x in driving_times.keys() else 0)
main_agent_df['inline'] = main_agent_df.AgentID.apply(lambda x: inline_times[x] if x in inline_times.keys() else 0)

In [52]:
charging_times

{1: 88,
 3: 37,
 4: 145,
 5: 55,
 7: 99,
 9: 107,
 10: 167,
 14: 87,
 16: 75,
 18: 87,
 19: 35,
 22: 73,
 23: 99,
 25: 87,
 26: 100,
 30: 146,
 31: 26,
 32: 100,
 36: 86,
 38: 97,
 39: 19,
 44: 87,
 49: 146,
 50: 46,
 51: 55,
 53: 167,
 54: 23,
 56: 72,
 59: 114,
 63: 99,
 64: 30,
 65: 52,
 66: 153,
 68: 35,
 69: 22,
 70: 97,
 73: 40,
 75: 19,
 76: 62,
 79: 30,
 83: 96,
 88: 55,
 92: 27,
 93: 105,
 94: 2,
 95: 75,
 96: 101,
 97: 49,
 98: 104,
 100: 124,
 101: 84,
 104: 93,
 105: 99,
 106: 73,
 107: 6,
 109: 95,
 111: 55,
 112: 27}

In [53]:
main_agent_df

,AgentID,Step,vessel_route,battery_size,generated_at,charging,inline,driving
0,0,8,"(NLTNZ, NLBRS, 0)",2300,7,0,0,0
1,1,21,"(NLNIJ, NLWAS, 0)",4750,20,0,0,0
2,2,32,"(NLDOR, NLRTM, 0)",4750,31,0,0,0
3,3,84,"(NLAMS, NLUTC, 1)",4750,83,0,0,0
4,4,84,"(NLNIJ, NLABL, 0)",6717,83,0,0,0
...,...,...,...,...,...,...,...,...
357,8868191,0,0,0,0,0,0,0
358,8868208,0,0,0,0,0,0,0
359,8868222,0,0,0,0,0,0,0
360,8868266,0,0,0,0,0,0,0


In [54]:
main_agent_df['charging'] = main_agent_df.AgentID.apply(lambda x: charging_times[x] if x in charging_times.keys() else 0)
main_agent_df['driving'] = main_agent_df.AgentID.apply(lambda x: driving_times[x] if x in driving_times.keys() else 0)
main_agent_df['inline'] = main_agent_df.AgentID.apply(lambda x: inline_times[x] if x in inline_times.keys() else 0)

In [55]:
main_agent_df

,AgentID,Step,vessel_route,battery_size,generated_at,charging,inline,driving
0,0,8,"(NLTNZ, NLBRS, 0)",2300,7,0,0,121
1,1,21,"(NLNIJ, NLWAS, 0)",4750,20,88,0,228
2,2,32,"(NLDOR, NLRTM, 0)",4750,31,0,0,130
3,3,84,"(NLAMS, NLUTC, 1)",4750,83,37,0,278
4,4,84,"(NLNIJ, NLABL, 0)",6717,83,145,0,578
...,...,...,...,...,...,...,...,...
357,8868191,0,0,0,0,0,0,0
358,8868208,0,0,0,0,0,0,0
359,8868222,0,0,0,0,0,0,0
360,8868266,0,0,0,0,0,0,0


## Old analysis

In [20]:
c.reset_index()

,index,Step,AgentID,vessel_status,vessel_route,battery_size,battery_level,generated_at,removed_at,station_status,status,at_station
0,0,0,"('30986055', '8867600')",0,0,0,0.000000,0,0.0,NaN,0,0
1,1,0,"('30986055', '8865756')",0,0,0,0.000000,0,0.0,NaN,0,0
2,2,0,"('8866889', '8863272')",0,0,0,0.000000,0,0.0,NaN,0,0
3,3,0,"('8866889', '131')",0,0,0,0.000000,0,0.0,NaN,0,0
4,4,0,"('30984595', '8860725')",0,0,0,0.000000,0,0.0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
377055,377327,1440,105,"[charging, 8865003]","(NLMOE, NLAPN, 1)",4750,739.962956,1281,NaN,NaN,charging,8865003
377056,377328,1440,108,driving,"(NLAMS, NLUTC, 1)",1000,915.000000,1328,NaN,NaN,driving,driving
377057,377329,1440,110,"[charging, 8866686]","(NLRTM, NLTNZ, 0)",1450,247.466969,1363,NaN,NaN,charging,8866686
377058,377330,1440,111,driving,"(NLFAR, NLEEM, 0)",4750,950.000000,1380,NaN,NaN,driving,driving


In [56]:
list(c.loc[0].index)

['charging116', 'charging160', 'charging8865412', 'driving', 'inline']

In [ ]:
def process_row(x):
    agentid = x.AgentID
    status = x.vessel_status
    count = x.id
    

In [12]:
c.loc[c.AgentID == 26].groupby('vessel_status').count()

,Step,AgentID,id,vessel_route,battery_size,battery_level,generated_at,removed_at,station_status
vessel_status,,,,,,,,,
charging122,44,44,44,44,44,44,44,0,0
driving,155,155,155,155,155,155,155,0,0
inline,97,97,97,97,97,97,97,0,0


In [14]:
c.loc[c.AgentID == 45].groupby('vessel_status').count()

,Step,AgentID,id,vessel_route,battery_size,battery_level,generated_at,removed_at,station_status
vessel_status,,,,,,,,,
driving,143,143,143,143,143,143,143,0,0


In [ ]:
c.loc[c.AgentID == 1]

In [ ]:
a = agent_data.loc[agent_data.station_status==1].reset_index().sort_values(by='Step')
a.groupby(['AgentID']).count()

In [ ]:
non_zero_flows[('NLRTM', 'NLMOE', 0)]

In [7]:
paths[('NLRTM', 'NLMOE', 0)]

['8863306',
 '8864009',
 '8863742',
 '8867500',
 '8865140',
 '8866686',
 '8863048',
 '8866570',
 '8863131',
 '8865003',
 '8863665',
 '8862663',
 '8860725',
 '30984595']

In [14]:
b = paths[('NLGOR', 'NLRTM', 0)]

In [15]:
b = b[::-1]

In [16]:
for i in range(1):
    print(i)

0


In [17]:
a1 =0
for i in range(len(b)-1):
    a = nx.dijkstra_path_length(G, b[i], b[i+1], weight='length_m')
    a1 += a
    print(a, a1)

651.3288639667403 651.3288639667403
6345.434042802258 6996.762906768998
3010.574501261619 10007.337408030617
4548.7114559023 14556.048863932916
1910.6073697711045 16466.65623370402
4354.183792219703 20820.840025923724
2859.5103864259836 23680.350412349708
831.6178353275283 24511.968247677236
584.9264533641734 25096.89470104141
24037.539243228617 49134.43394427003
7912.849145725791 57047.28308999582
1655.7769099938282 58703.05999998965
690.3248762319221 59393.38487622157


In [ ]:
a1

In [18]:
path_lengths[('NLRTM', 'NLAPN', 0)]

46113.84265717132

In [19]:
path_lengths[('NLTNZ', 'NLVLI', 0)]

27535.141001402484

In [ ]:
paths[('NLKGZ', 'NLAMS', 0)]

In [ ]:
agent_data.reset_index().loc[agent_data.reset_index().AgentID.str.len()<4]

In [ ]:
agent_data.reset_index(inplace=True)

In [ ]:

agent_data.loc[agent_data.station_status == 1]

In [69]:
agent_data.loc[agent_data.Vessel_status!=0]

AttributeError: 'DataFrame' object has no attribute 'Vessel_status'

In [ ]:
df_abm.loc[df_abm.name]

In [ ]:
o = {}
for key, item in optimal_flows.items():
    if item != 0:
        a = (re.sub('''["'_']''', "", key[16:35]).split(','))
        a = tuple([a[0], a[1], int(a[2])])
        o[a] = {'combinations':[], 'flows':[]}

for key, item in optimal_flows.items():
    if item != 0:
        a = (re.sub('''["'_']''', "", key[16:35]).split(','))
        a = tuple([a[0], a[1], int(a[2])])
        o[a]['combinations'].append(key[40:-4].split("',_'"))
        o[a]['flows'].append(item)
        print(a)
        print(key[40:-4].split("',_'"))


In [ ]:
o

In [ ]:
a = o[('NLAMS', 'NLUTC', 1)]
combi = np.random.choice(a=np.arange(len(a)), size=1, replace=False, p=[0.5,0.5])

In [ ]:
a = [1,2,3]
a = [i/sum(a) for i in a]

In [ ]:
a

In [ ]:
df_abm.loc[df_abm.name == '8864005']

In [ ]:
updated_feasible_combinations = {}
for key, combinations in feasible_combinations.items():
    updated_feasible_combinations[key] = []
    if len(combinations) > 1:
        for combi in combinations:
            # print(combi)
            feasible = True
            for node in combi:
                if not node in charging_points:
                    feasible = False
                    break
            if feasible:
                updated_feasible_combinations[key].append(combi)
    else:
        feasible = True
        for node in combinations:
            if not node in charging_points:
                feasible = False
                break
        if feasible:
            updated_feasible_combinations[key].append(combinations)



In [ ]:
a = [1]
a[0]

In [ ]:
# extract data
df_links = nx.to_pandas_edgelist(G)
df_nodes = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

In [ ]:
for _, row in df_abm.iterrows():  # index, row in ...
    # create agents according to model_type

    model_type = row['model_type']
    agent = None

    name = row['name']
    if pd.isna(name):
        name = ""
        print("error, nameless entry")
    else:
        name = name
    break

In [ ]:
a =[1,3,34,6]
a[::-1]

In [ ]:
df_random.trip_count.iloc[0]

In [ ]:
%%time
type_list = list(df_random.iloc[:, 4:-2])
for hour in range(24):
    df_1 = df_random.loc[(df_random.hour == hour)]
    for timestep in range(60):
        for harbour in list(set(list(df_1.origin.unique()) + list(df_1.destination.unique()))):
            a = df_1.loc[(df_1.origin==harbour)|(df_1.destination == harbour)]
            for i,j in enumerate(a.index):
                # chance that a vessel is generated is equal to 1/2 (either spawn at origin or dest) * trip_count/365
                # (because trip_count is yearly) *(time_step/hours)
                if 0.5 * (df_1.trip_count[j]/365) * (1/60) >= np.random.random():
                    # determine vessel type
                    prob = list(a.iloc[i, 4:-2].values / a.iloc[i, 4:-2].sum())
                    to_pick = type_list
                    ship_type = np.random.choice(a=to_pick, size=1, replace=True, p=prob)
                    # randomly draw ship types for each OD pair
                    print(ship_type, "Vessel departed at", df_1.origin[j], hour, ':', timestep, "heading to", df_1.destination[j], "via route", df_1.key[j])


In [ ]:
type_list

In [ ]:
def random_vessel_generator(df_prob, load=1):
    """ Function to generate random vessel data.
    Parameters
    ----------
    df_prob: pd.DataFrame
    This dataframe must contain historical travel data
    load: float
    This float is equal to the simulated occupation of the network and may be read as the part of the traffic that is
    electric.
    """
    df_prob = df_prob.loc[df_prob.trip_count != 0]
    df_prob.reset_index(inplace=True, drop=True)
    df_prob = df_prob.fillna(0)

    # create dict to store random prob based values later on
    main_dict = {i: [] for i in df_prob.columns}

    # copy origin and destination from original df
    main_dict['origin'] = []
    main_dict['destination'] = []
    main_dict['route_v'] = []
    main_dict['hour'] = []
    main_dict['trip_count'] = []

    # fill nan values and create type list
    type_list = list(df_prob.iloc[:, 4:-1])
    for i in range(24):
        df_temp = df_prob.loc[df_prob.hour == i]
        df_temp = df_temp.reset_index(drop=True)
        total = df_temp.trip_count.sum()
        prob_r = [i / total for i in df_temp.trip_count]
        to_pick_r = list(np.arange(0, len(df_temp)))
        count_r = total * 1
        # randomly draw ODs
        rand_routes = np.random.choice(a=to_pick_r, size=round(count_r), replace=True, p=prob_r)
        unique_r, counts_r = np.unique(rand_routes, return_counts=True)
        temp_type_dict_r = dict(zip(unique_r, counts_r))

        for pair in df_temp.index:
            # store trip count data previous draw
            if pair in temp_type_dict_r.keys():
                main_dict['trip_count'].append(temp_type_dict_r[pair])
                main_dict['origin'].append(df_temp.origin[pair])
                main_dict['destination'].append(df_temp.destination[pair])
                main_dict['route_v'].append(df_temp.route_v[pair])
                main_dict['hour'].append(i)
                # print(main_dict['trip_count'])
                # determine parameters to randomly draw ship types for each OD pair
                prob = list(df_temp.iloc[pair, 4:-1].values / df_temp.iloc[pair, 4:-1].sum())
                to_pick = type_list
                # goes wrong now because may be zero ...
                count = temp_type_dict_r[pair]

                # randomly draw ship types for each OD pair
                rand_vessels = np.random.choice(a=to_pick, size=round(count), replace=True, p=prob)
                unique, counts = np.unique(rand_vessels, return_counts=True)
                temp_type_dict = dict(zip(unique, counts))
                for key in type_list:
                    if key in temp_type_dict.keys():
                        main_dict[key].append(temp_type_dict[key])
                    else:
                        main_dict[key].append(0)
    break

    df_return = pd.DataFrame.from_dict(main_dict)

    return df_return

In [ ]:
a1 = random_vessel_generator(df_ivs, 1)

In [ ]:
a1